In [4]:
import cv2
import os
import numpy as np
from PIL import Image
import pyautogui
import ctypes
import os
import datetime
import time
import sys

In [5]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [3]:
def check_path(path):            
    dir = os.path.dirname(path)  
    if not os.path.exists(dir):
        os.makedirs(dir,exist_ok=True)

vid_cam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
face_id = 1
count = 0
check_path("dataset/")

while(True):
    _,image_frame = vid_cam.read()       
    
    gray = cv2.cvtColor(image_frame, cv2.COLOR_BGR2GRAY) 
    
    faces = face_cascade.detectMultiScale(gray, 1.4, 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(image_frame, (x,y), (x+w,y+h), (255,0,0), 2) 
        
        count += 1              

        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w]) 

        cv2.imshow('Creating Dataset', image_frame)

    if cv2.waitKey(100) & 0xFF == 27:                  
        break

    elif count>100:                                    
        break

vid_cam.release()                                    

cv2.destroyAllWindows()                            

In [ ]:
def assure_path_exists(path):          
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)


def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples = []
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = face_cascade.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)


    return faceSamples,ids  

faces,ids = getImagesAndLabels('dataset/')

recognizer.train(faces, np.array(ids))

assure_path_exists('trainer/') 
recognizer.save('trainer/trainer.yml')


In [9]:
recognizer.read('trainer/trainer.yml')
cam = cv2.VideoCapture(0)
now = datetime.datetime.now().second
counter_wrong = 0 
counter_correct = 0
while True:
    now1 = datetime.datetime.now()
    now1 = now1.second
    if(now1 > now + 10):
        cam.release()
        cv2.destroyAllWindows()
        ctypes.windll.user32.LockWorkStation()
        sys.exit()

    ret, im =cam.read()
    
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3,5)

    for(x,y,w,h) in faces:

        cv2.rectangle(im, (x-20,y-20), (x+w+20,y+h+20), (0,255,0), 4)

        Id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        if(confidence>80):
            counter_wrong += 1
            print("Wrong")
            Id = "Unknown + {0:.2f}%".format(round(100 - confidence, 2)) 
            print(confidence)
            print("counter_wrong - " + str(counter_wrong))
            cv2.rectangle(im, (x-22,y-90), (x+w+22, y-22), (0,0,255), -1)
            cv2.putText(im, str(Id), (x,y-40), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,0), 2)

        else:
            Id = "Khan + {0:.2f}%".format(round(100 - confidence, 2)) 
            print("Verified")
            print(confidence)
            cv2.rectangle(im, (x-22,y-90), (x+w+22, y-22), (255,255,255), -1)
            cv2.putText(im, str(Id), (x,y-40), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,0), 2)

        if(counter_wrong == 3):
            pyautogui.moveTo(48,748)
            pyautogui.click(48,748)
            pyautogui.typewrite("Hello Stranger!")
            cam.release()
            cv2.destroyAllWindows()
            ctypes.windll.user32.LockWorkStation()
            sys.exit()
        
        if(counter_wrong == 6):
            cam.release()
            cv2.destroyAllWindows()
            ctypes.windll.user32.LockWorkStation()
            sys.exit()

    cv2.imshow("webcam",im)

    if cv2.waitKey(10) & 0xFF == ord('q'):     
        break

cam.release()

cv2.destroyAllWindows()

SystemExit: 

c:\Users\pavan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
